In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.applications import ResNet50

# Definir tamanho das imagens
img_height, img_width = 224, 224
data='C:/Users/ID MADEIRA/Documents/Projetos/seminario2/juntas'
# Carregar o modelo de transferência de aprendizado pré-treinado (usando ResNet50 como exemplo)
base_model = ResNet50(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet')

# Congelar o modelo base
base_model.trainable = False

# Adicionar novas camadas ao modelo
inputs = layers.Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(12, activation='softmax')(x)

model = keras.Model(inputs, outputs)

# Definir aumento de dados para conjunto de treinamento e validação
datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range=35,
                             horizontal_flip=True,
                             vertical_flip=True,
                             fill_mode='nearest',
                             validation_split=0.3,
                             zoom_range=0.2,
                             preprocessing_function=keras.applications.resnet.preprocess_input)

# Gerar lotes de imagens de treinamento e validação
train_generator = datagen.flow_from_directory(
    data,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = datagen.flow_from_directory(
    data,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

# Adicionar EarlyStopping para prevenir overfitting
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    verbose=1)

# Treinar o modelo (fase 1)
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=[keras.metrics.CategoricalAccuracy()])

history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=[early_stopping])

# Descongelar todas as camadas para ajuste fino
base_model.trainable = True

# Ajustar hiperparâmetros para o ajuste fino
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=[keras.metrics.CategoricalAccuracy()])

# Treinar o modelo (fase 2)
history_fine = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[early_stopping])


test_generator = test_datagen.flow_from_directory(
    data,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical')


# Avaliar o modelo no conjunto de teste
test_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=keras.applications.resnet.preprocess_input)

test_loss, test_acc = model.evaluate(test_generator)

# Prever as classes para o conjunto de teste
y_pred = model.predict(test_generator)
y_true = np.argmax(test_generator.classes, axis=1)

# Matriz de Confusão
conf_matrix = confusion_matrix(y_true, np.argmax(y_pred, axis=1))
print("Matriz de Confusão:")
print(conf_matrix)

# Relatório de Classificação
class_report = classification_report(y_true, np.argmax(y_pred, axis=1))
print("Relatório de Classificação:")
print(class_report)

# Curva ROC
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(12):  # número de classes
    fpr[i], tpr[i], _ = roc_curve(y_true == i, y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotar a curva ROC para cada classe
plt.figure(figsize=(10, 6))
for i in range(12):
    plt.plot(fpr[i], tpr[i], label=f'Classe {i} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)  # Linha diagonal para referência
plt.xlabel('Taxa de Falso Positivo')
plt.ylabel('Taxa de Verdadeiro Positivo')
plt.title('Curva ROC por Classe')
plt.legend(loc="lower right")
plt.show()
